In [1]:
import json
import tensorflow as tf
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1


In [12]:
column = df["humor_rating"]
max_value = column.max()



print(max_value)

4.0


In [4]:
# Separate the json into sentences and labels
sentences = []
labels = []

#
    
for i,l,text,humour,rating,controversy,offense in df.itertuples():
    sentences.append(text)
    labels.append(humour)
    
print(pd.DataFrame({'senetence' : sentences[0:10], 'label':labels[0:10]}))

                                           senetence  label
0  TENNESSEE: We're the best state. Nobody even c...      1
1  A man inserted an advertisement in the classif...      1
2  How many men does it take to open a can of bee...      1
3  Told my mom I hit 1200 Twitter followers. She ...      1
4  Roses are dead. Love is fake. Weddings are bas...      1
5  'Trabajo,' the Spanish word for work, comes fr...      0
6  I enrolled on some skill training and extra cu...      0
7  ME: I'm such an original. Truly one of a kind....      1
8  Men who ejaculated 21 times or more a month ha...      0
9  I got REALLY angry today and it wasn't about n...      0


In [5]:
training_size = round(len(sentences) * .75)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]


# Setting tokenizer properties
vocab_size = 20000
oov_tok = "<oov>"

# Fit the tokenizer on Training data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

# Setting the padding properties
max_length = 60
trunc_type='post'
padding_type='post'


# Creating padded sequences from train and test data
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [6]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
embedding_dim = 64

#embedding_dim = 16


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 64)            1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 1,281,585
Trainable params: 1,281,585
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Converting the lists to numpy arrays for Tensorflow 2.x
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)


# Training the model
num_epochs = 15

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Train on 6000 samples, validate on 2000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 - 24s - loss: 0.6374 - acc: 0.6307 - val_loss: 0.5632 - val_acc: 0.7240
Epoch 2/15
 - 6s - loss: 0.4177 - acc: 0.8283 - val_loss: 0.3703 - val_acc: 0.8450
Epoch 3/15
 - 6s - loss: 0.2601 - acc: 0.9030 - val_loss: 0.3317 - val_acc: 0.8615
Epoch 4/15
 - 5s - loss: 0.1764 - acc: 0.9383 - val_loss: 0.3294 - val_acc: 0.8640
Epoch 5/15
 - 5s - loss: 0.1226 - acc: 0.9642 - val_loss: 0.3368 - val_acc: 0.8670
Epoch 6/15
 - 5s - loss: 0.0855 - acc: 0.9780 - val_loss: 0.3683 - val_acc: 0.8630
Epoch 7/15
 - 5s - loss: 0.0609 - acc: 0.9857 - val_loss: 0.3871 - val_acc: 0.8640
Epoch 8/15
 - 5s - loss: 0.0440 - acc: 0.9900 - val_loss: 0.4100 - val_acc: 0.8570
Epoch 9/15
 - 6s - loss: 0.0327 - acc: 0.9943 - val_loss: 0.4319 - val_acc: 0.8540
Epoch 10/15
 - 6s - loss: 0.0252 - acc: 0.9963 - val_loss: 0.4977 - val_acc: 0.8520
Epoch 11/15
 - 6s - loss: 0.0192 - acc: 0.9970 - val_loss: 0.4853 - val_

In [30]:
import numpy as np 
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import scipy.stats
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
df['humor_controversy'] = df['humor_controversy'].replace(np.nan, 0)
df['offense_rating'] = df['offense_rating'].replace(np.nan, 0)
df['humor_rating'] = df['humor_rating'].replace(np.nan, 0)
df['is_humor'] = df['is_humor'].replace(np.nan, 0)

humor = df['is_humor'].values
df1 = pd.DataFrame()
#df['humor_rating'] = df['humor_rating'].replace(np.nan, 0)

def Score_Predict2(df,model):
    df['score']=df['text'].apply(lambda text: model.predict(pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=max_length, padding=padding_type, truncating=trunc_type)))
    df['score'] = df['score'].replace(np.nan, 0)
    df['score'] = df['score'].apply(lambda s: s[0][0])
    df['My_rate']=df['score'].apply(lambda s: s*4)
    df['new_rate']=df['My_rate'] - df['humor_controversy']-df['offense_rating']
    df['new_rate'] = df['new_rate'].replace(np.nan, 0)
    df['new_rate'] = df['new_rate']*df['is_humor']
    actual = df['humor_rating'].values
    pred = df['new_rate'].values
    mae = metrics.mean_absolute_error(actual, pred)
    print("MAE1:",mae)
    print("Pearson Corr.1 :",scipy.stats.pearsonr(actual, pred)[0] )
    print("Spearman corr.1 :",scipy.stats.spearmanr(actual, pred)[0])
    print("kendall.1 :",scipy.stats.kendalltau(actual, pred)[0])
    #2nd method
    features = df[['humor_controversy','offense_rating','score','is_humor']].values
    labels1 = df['humor_rating'].values
    # getting training and testing data
    features_train, features_test, labels_train1, labels_test1 = train_test_split(features, labels1, test_size = .75, random_state = 0)
    scaler=MinMaxScaler()
    scaler.fit(features_train)
    features_train=scaler.transform(features_train)
    features_test=scaler.transform(features_test)
    model1=Sequential()
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(1))
    model1.compile(loss='mse',optimizer='adam')#,metrics=['accuracy'])
    history=model1.fit(x=features_train,y=labels_train1,epochs=30,validation_data=(features_test, labels_test1),verbose=2)
    #humor_pred=model1.predict(features_test)
    yhat = model1.predict(features, verbose=0)  
    nice=[]
    for i in yhat:
        nice.append(i[0])
    pred1 = [humor[i] * nice[i] for i in range(len(nice))]    
    mae2 = metrics.mean_absolute_error(actual, pred1)
    print("MAE2:",mae2)
    print("Pearson Corr.2 :",scipy.stats.pearsonr(actual, pred1)[0] )
    print("Spearman corr.2 :",scipy.stats.spearmanr(actual, pred1)[0])
    print("kendall.2 :",scipy.stats.kendalltau(actual, pred1)[0])
    d = {"original": actual, "1st": pred, "2nd": pred1}
    df1 = pd.DataFrame(d)
    print(df1 )


    

In [31]:
Score_Predict2(df,model)

MAE1: 0.6536443962625156
Pearson Corr.1 : 0.7587482851006877
Spearman corr.1 : 0.7331326649735217
kendall.1 : 0.5517137776608328
Train on 2000 samples, validate on 6000 samples
Epoch 1/30
 - 3s - loss: 1.3027 - val_loss: 0.6628
Epoch 2/30
 - 0s - loss: 0.5062 - val_loss: 0.3898
Epoch 3/30
 - 0s - loss: 0.3979 - val_loss: 0.3369
Epoch 4/30
 - 0s - loss: 0.3489 - val_loss: 0.2944
Epoch 5/30
 - 0s - loss: 0.3100 - val_loss: 0.2620
Epoch 6/30
 - 0s - loss: 0.2804 - val_loss: 0.2401
Epoch 7/30
 - 0s - loss: 0.2594 - val_loss: 0.2232
Epoch 8/30
 - 0s - loss: 0.2442 - val_loss: 0.2101
Epoch 9/30
 - 0s - loss: 0.2312 - val_loss: 0.2013
Epoch 10/30
 - 0s - loss: 0.2228 - val_loss: 0.1944
Epoch 11/30
 - 0s - loss: 0.2149 - val_loss: 0.1911
Epoch 12/30
 - 0s - loss: 0.2113 - val_loss: 0.1866
Epoch 13/30
 - 0s - loss: 0.2059 - val_loss: 0.1835
Epoch 14/30
 - 0s - loss: 0.2027 - val_loss: 0.1808
Epoch 15/30
 - 0s - loss: 0.2008 - val_loss: 0.1793
Epoch 16/30
 - 0s - loss: 0.1980 - val_loss: 0.1784


In [32]:
df

,id,text,is_humor,humor_rating,humor_controversy,offense_rating,score,My_rate,new_rate
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20,9.957010e-01,3.982804e+00,2.782804
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10,9.997738e-01,3.999095e+00,1.899095
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40,9.998106e-01,3.999243e+00,1.599243
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00,9.998872e-01,3.999549e+00,2.999549
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10,9.986625e-01,3.994650e+00,3.894650
...,...,...,...,...,...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,0.00,0.0,0.25,1.289564e-06,5.158257e-06,-0.000000
7996,7997,Why are aspirins white? Because they work sorry,1,1.33,0.0,3.85,9.868304e-01,3.947322e+00,0.097322
7997,7998,"Today, we Americans celebrate our independence...",1,2.55,0.0,0.00,1.733924e-01,6.935695e-01,0.693570
7998,7999,How to keep the flies off the bride at an Ital...,1,1.00,0.0,3.00,9.998449e-01,3.999380e+00,0.999380


In [33]:
#RNN
from tensorflow.keras.callbacks import ReduceLROnPlateau 

model_rnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.SimpleRNN(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 60, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 60, 32)            3104      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 17        
Total para

In [34]:
history_rnn = model_rnn.fit(training_padded, training_labels, batch_size=32, epochs=15, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Epoch 1/15
6000/6000 [==============================] - 12s 2ms/sample - loss: 0.6769 - acc: 0.5868 - val_loss: 0.6605 - val_acc: 0.6140
Epoch 2/15
6000/6000 [==============================] - 9s 2ms/sample - loss: 0.6554 - acc: 0.6185 - val_loss: 0.5741 - val_acc: 0.6200
Epoch 3/15
6000/6000 [==============================] - 9s 2ms/sample - loss: 0.5457 - acc: 0.7435 - val_loss: 0.4984 - val_acc: 0.7810
Epoch 4/15
6000/6000 [==============================] - 9s 1ms/sample - loss: 0.4712 - acc: 0.8003 - val_loss: 0.4840 - val_acc: 0.7760
Epoch 5/15
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.4032 - acc: 0.8315 - val_loss: 0.4423 - val_acc: 0.8045
Epoch 6/15
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.3316 - acc: 0.8748 - val_loss: 0.4080 - val_acc: 0.8265
Epoch 7/15
6000/6000 [==============================] - 8s 1ms/sample - loss: 0.2722 - acc: 0.8998 - val_loss: 0.4152 - val_acc: 0.8240

In [35]:
Score_Predict2(df,model_rnn)

MAE1: 0.6543699146240624
Pearson Corr.1 : 0.7523786622062567
Spearman corr.1 : 0.7281117880672627
kendall.1 : 0.5491269990936982
Train on 2000 samples, validate on 6000 samples
Epoch 1/30
 - 2s - loss: 3.2246 - val_loss: 2.9667
Epoch 2/30
 - 0s - loss: 2.7651 - val_loss: 2.4296
Epoch 3/30
 - 0s - loss: 2.1226 - val_loss: 1.6918
Epoch 4/30
 - 0s - loss: 1.3196 - val_loss: 0.8795
Epoch 5/30
 - 0s - loss: 0.6118 - val_loss: 0.3528
Epoch 6/30
 - 0s - loss: 0.2836 - val_loss: 0.2098
Epoch 7/30
 - 0s - loss: 0.2119 - val_loss: 0.1844
Epoch 8/30
 - 0s - loss: 0.1958 - val_loss: 0.1769
Epoch 9/30
 - 0s - loss: 0.1905 - val_loss: 0.1748
Epoch 10/30
 - 0s - loss: 0.1885 - val_loss: 0.1729
Epoch 11/30
 - 0s - loss: 0.1873 - val_loss: 0.1735
Epoch 12/30
 - 0s - loss: 0.1868 - val_loss: 0.1719
Epoch 13/30
 - 0s - loss: 0.1876 - val_loss: 0.1720
Epoch 14/30
 - 0s - loss: 0.1859 - val_loss: 0.1722
Epoch 15/30
 - 0s - loss: 0.1847 - val_loss: 0.1771
Epoch 16/30
 - 0s - loss: 0.1868 - val_loss: 0.1722


In [36]:
df

,id,text,is_humor,humor_rating,humor_controversy,offense_rating,score,My_rate,new_rate
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20,0.919409,3.677636,2.477636
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10,0.999238,3.996951,1.896951
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40,0.995887,3.983549,1.583549
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00,0.999528,3.998111,2.998111
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10,0.986423,3.945691,3.845691
...,...,...,...,...,...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,0.00,0.0,0.25,0.000504,0.002014,-0.000000
7996,7997,Why are aspirins white? Because they work sorry,1,1.33,0.0,3.85,0.995001,3.980004,0.130004
7997,7998,"Today, we Americans celebrate our independence...",1,2.55,0.0,0.00,0.017861,0.071444,0.071444
7998,7999,How to keep the flies off the bride at an Ital...,1,1.00,0.0,3.00,0.999713,3.998852,0.998852


In [37]:
model_lstm_avg = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_lstm_avg.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm_avg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 60, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 32)            12416     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 32)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 17        
Total para

In [38]:
history_lstm_avg = model_lstm_avg.fit(training_padded, training_labels, batch_size=32, epochs=num_epochs, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Epoch 1/15
6000/6000 [==============================] - 23s 4ms/sample - loss: 0.5737 - acc: 0.6975 - val_loss: 0.4270 - val_acc: 0.8160
Epoch 2/15
6000/6000 [==============================] - 18s 3ms/sample - loss: 0.3644 - acc: 0.8623 - val_loss: 0.3580 - val_acc: 0.8560
Epoch 3/15
6000/6000 [==============================] - 18s 3ms/sample - loss: 0.2353 - acc: 0.9243 - val_loss: 0.4175 - val_acc: 0.8545
Epoch 4/15
6000/6000 [==============================] - 19s 3ms/sample - loss: 0.1602 - acc: 0.9538 - val_loss: 0.4328 - val_acc: 0.8530
Epoch 5/15
6000/6000 [==============================] - 18s 3ms/sample - loss: 0.1072 - acc: 0.9738 - val_loss: 0.4387 - val_acc: 0.8550
Epoch 6/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.0943 - acc: 0.9792 - val_loss: 0.4728 - val_acc: 0.8550
Epoch 7/15
6000/6000 [==============================] - 18s 3ms/sample - loss: 0.0916 - acc: 0.9778 - val_loss: 0.4727 - val_acc: 

In [39]:
Score_Predict2(df,model_lstm_avg)

MAE1: 0.6552756379737426
Pearson Corr.1 : 0.7487524373766052
Spearman corr.1 : 0.7241805223880627
kendall.1 : 0.5471831294158913
Train on 2000 samples, validate on 6000 samples
Epoch 1/30
 - 3s - loss: 2.9971 - val_loss: 2.4687
Epoch 2/30
 - 0s - loss: 1.9633 - val_loss: 1.3316
Epoch 3/30
 - 0s - loss: 0.9895 - val_loss: 0.6680
Epoch 4/30
 - 0s - loss: 0.5220 - val_loss: 0.3798
Epoch 5/30
 - 0s - loss: 0.3308 - val_loss: 0.2685
Epoch 6/30
 - 0s - loss: 0.2589 - val_loss: 0.2265
Epoch 7/30
 - 0s - loss: 0.2307 - val_loss: 0.2073
Epoch 8/30
 - 0s - loss: 0.2159 - val_loss: 0.1982
Epoch 9/30
 - 0s - loss: 0.2085 - val_loss: 0.1939
Epoch 10/30
 - 0s - loss: 0.2038 - val_loss: 0.1888
Epoch 11/30
 - 0s - loss: 0.2014 - val_loss: 0.1847
Epoch 12/30
 - 0s - loss: 0.2002 - val_loss: 0.1826
Epoch 13/30
 - 0s - loss: 0.1980 - val_loss: 0.1817
Epoch 14/30
 - 0s - loss: 0.1965 - val_loss: 0.1811
Epoch 15/30
 - 0s - loss: 0.1950 - val_loss: 0.1788
Epoch 16/30
 - 0s - loss: 0.1940 - val_loss: 0.1783


In [40]:
df

,id,text,is_humor,humor_rating,humor_controversy,offense_rating,score,My_rate,new_rate
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20,0.969633,3.878530,2.678530
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10,0.990838,3.963352,1.863352
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40,0.985280,3.941121,1.541121
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00,0.993814,3.975257,2.975257
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10,0.986595,3.946382,3.846382
...,...,...,...,...,...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,0.00,0.0,0.25,0.009065,0.036260,-0.000000
7996,7997,Why are aspirins white? Because they work sorry,1,1.33,0.0,3.85,0.992849,3.971397,0.121397
7997,7998,"Today, we Americans celebrate our independence...",1,2.55,0.0,0.00,0.015204,0.060814,0.060814
7998,7999,How to keep the flies off the bride at an Ital...,1,1.00,0.0,3.00,0.990699,3.962796,0.962796


In [41]:
from tensorflow.keras.callbacks import ReduceLROnPlateau 

model_lstm1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_lstm1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 60, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60, 32)            12416     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 33        
Total params: 1,292,449
Trainable params: 1,292,449
Non-trainable params: 0
_________________________________________________________________


In [42]:
history_lstm1 = model_lstm1.fit(training_padded, training_labels, batch_size=32, epochs=num_epochs, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Epoch 1/15
6000/6000 [==============================] - 20s 3ms/sample - loss: 0.5695 - acc: 0.6920 - val_loss: 0.4079 - val_acc: 0.8195
Epoch 2/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.3170 - acc: 0.8722 - val_loss: 0.3394 - val_acc: 0.8540
Epoch 3/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.1881 - acc: 0.9347 - val_loss: 0.3706 - val_acc: 0.8550
Epoch 4/15
6000/6000 [==============================] - 18s 3ms/sample - loss: 0.1088 - acc: 0.9658 - val_loss: 0.4099 - val_acc: 0.8415
Epoch 5/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.0584 - acc: 0.9835 - val_loss: 0.4055 - val_acc: 0.8445
Epoch 6/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.0506 - acc: 0.9863 - val_loss: 0.4361 - val_acc: 0.8460
Epoch 7/15
6000/6000 [==============================] - 17s 3ms/sample - loss: 0.0481 - acc: 0.9868 - val_loss: 0.4376 - val_acc: 

In [43]:
Score_Predict2(df,model_lstm1)

MAE1: 0.6541895267407176
Pearson Corr.1 : 0.7480504976252162
Spearman corr.1 : 0.7250628800064474
kendall.1 : 0.5473857281360217
Train on 2000 samples, validate on 6000 samples
Epoch 1/30
 - 3s - loss: 3.2443 - val_loss: 2.6736
Epoch 2/30
 - 0s - loss: 2.2390 - val_loss: 1.7790
Epoch 3/30
 - 0s - loss: 1.5147 - val_loss: 1.2383
Epoch 4/30
 - 0s - loss: 1.0919 - val_loss: 0.8935
Epoch 5/30
 - 0s - loss: 0.7673 - val_loss: 0.5918
Epoch 6/30
 - 0s - loss: 0.5266 - val_loss: 0.4193
Epoch 7/30
 - 0s - loss: 0.4025 - val_loss: 0.3304
Epoch 8/30
 - 0s - loss: 0.3308 - val_loss: 0.2789
Epoch 9/30
 - 0s - loss: 0.2906 - val_loss: 0.2505
Epoch 10/30
 - 0s - loss: 0.2661 - val_loss: 0.2314
Epoch 11/30
 - 0s - loss: 0.2484 - val_loss: 0.2176
Epoch 12/30
 - 0s - loss: 0.2336 - val_loss: 0.2034
Epoch 13/30
 - 0s - loss: 0.2184 - val_loss: 0.1916
Epoch 14/30
 - 0s - loss: 0.2055 - val_loss: 0.1813
Epoch 15/30
 - 0s - loss: 0.1949 - val_loss: 0.1747
Epoch 16/30
 - 0s - loss: 0.1889 - val_loss: 0.1721


In [44]:
df

,id,text,is_humor,humor_rating,humor_controversy,offense_rating,score,My_rate,new_rate
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20,0.803353,3.213413,2.013413
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10,0.996359,3.985436,1.885436
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40,0.994951,3.979802,1.579802
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00,0.998251,3.993006,2.993006
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10,0.990869,3.963475,3.863475
...,...,...,...,...,...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,0.00,0.0,0.25,0.001270,0.005080,-0.000000
7996,7997,Why are aspirins white? Because they work sorry,1,1.33,0.0,3.85,0.993704,3.974817,0.124817
7997,7998,"Today, we Americans celebrate our independence...",1,2.55,0.0,0.00,0.033051,0.132206,0.132206
7998,7999,How to keep the flies off the bride at an Ital...,1,1.00,0.0,3.00,0.994729,3.978916,0.978916


In [45]:
#
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.keras.layers import Flatten, Conv2D
from tensorflow.keras.layers import Conv2D

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 60, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 128)           41088     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 250)               32250     
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 251       
Total para

In [46]:
history_cnn = model_cnn.fit(training_padded, training_labels, batch_size=32, epochs=num_epochs, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Epoch 1/15
6000/6000 [==============================] - 12s 2ms/sample - loss: 0.5125 - acc: 0.7325 - val_loss: 0.3470 - val_acc: 0.8495
Epoch 2/15
6000/6000 [==============================] - 10s 2ms/sample - loss: 0.2322 - acc: 0.9070 - val_loss: 0.3314 - val_acc: 0.8595
Epoch 3/15
6000/6000 [==============================] - 9s 2ms/sample - loss: 0.0765 - acc: 0.9742 - val_loss: 0.4057 - val_acc: 0.8435
Epoch 4/15
6000/6000 [==============================] - 10s 2ms/sample - loss: 0.0185 - acc: 0.9950 - val_loss: 0.5161 - val_acc: 0.8450
Epoch 5/15
6000/6000 [==============================] - 9s 2ms/sample - loss: 0.0053 - acc: 0.9993 - val_loss: 0.5138 - val_acc: 0.8465
Epoch 6/15
6000/6000 [==============================] - 10s 2ms/sample - loss: 0.0041 - acc: 0.9997 - val_loss: 0.5208 - val_acc: 0.8450
Epoch 7/15
6000/6000 [==============================] - 9s 2ms/sample - loss: 0.0035 - acc: 0.9998 - val_loss: 0.5216 - val_acc: 0.8

In [47]:
Score_Predict2(df,model_cnn)

MAE1: 0.6668321638230452
Pearson Corr.1 : 0.7482126771279378
Spearman corr.1 : 0.7235122194662413
kendall.1 : 0.5448200637707373
Train on 2000 samples, validate on 6000 samples
Epoch 1/30
 - 3s - loss: 3.1009 - val_loss: 2.4047
Epoch 2/30
 - 0s - loss: 1.8740 - val_loss: 1.2179
Epoch 3/30
 - 0s - loss: 0.7871 - val_loss: 0.4365
Epoch 4/30
 - 0s - loss: 0.4061 - val_loss: 0.3445
Epoch 5/30
 - 0s - loss: 0.3399 - val_loss: 0.2892
Epoch 6/30
 - 0s - loss: 0.2895 - val_loss: 0.2476
Epoch 7/30
 - 0s - loss: 0.2571 - val_loss: 0.2245
Epoch 8/30
 - 0s - loss: 0.2365 - val_loss: 0.2069
Epoch 9/30
 - 0s - loss: 0.2230 - val_loss: 0.1972
Epoch 10/30
 - 0s - loss: 0.2134 - val_loss: 0.1900
Epoch 11/30
 - 0s - loss: 0.2073 - val_loss: 0.1848
Epoch 12/30
 - 0s - loss: 0.2017 - val_loss: 0.1810
Epoch 13/30
 - 0s - loss: 0.1981 - val_loss: 0.1784
Epoch 14/30
 - 0s - loss: 0.1952 - val_loss: 0.1761
Epoch 15/30
 - 0s - loss: 0.1933 - val_loss: 0.1749
Epoch 16/30
 - 0s - loss: 0.1918 - val_loss: 0.1738


In [48]:
df

,id,text,is_humor,humor_rating,humor_controversy,offense_rating,score,My_rate,new_rate
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20,9.988213e-01,3.995285,2.795285
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10,9.999614e-01,3.999846,1.899846
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40,9.998510e-01,3.999404,1.599404
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00,9.999874e-01,3.999949,2.999949
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10,9.967354e-01,3.986942,3.886942
...,...,...,...,...,...,...,...,...,...
7995,7996,Lack of awareness of the pervasiveness of raci...,0,0.00,0.0,0.25,1.363690e-04,0.000545,-0.000000
7996,7997,Why are aspirins white? Because they work sorry,1,1.33,0.0,3.85,9.994358e-01,3.997743,0.147743
7997,7998,"Today, we Americans celebrate our independence...",1,2.55,0.0,0.00,2.944199e-03,0.011777,0.011777
7998,7999,How to keep the flies off the bride at an Ital...,1,1.00,0.0,3.00,9.999983e-01,3.999993,0.999993


In [50]:
model.save('My_embedding')
model_rnn.save('My_rnn')
model_lstm_avg.save('My_lstm_avg')
model_lstm1.save('My_lstm_max')
model_cnn.save('My_cnn')


In [11]:
df=pd.read_csv('train.csv')

In [9]:
from tensorflow.keras.callbacks import ReduceLROnPlateau 

model_bidir = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_bidir.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bidir.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 64)            24832     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,305,889
Tra

In [10]:

# Training the model
num_epochs = 5

history_bidir = model_bidir.fit(training_padded, training_labels, batch_size=32, epochs=num_epochs, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
6000/6000 [==============================] - 26s 4ms/sample - loss: 0.5421 - acc: 0.7090 - val_loss: 0.3773 - val_acc: 0.8445
Epoch 2/5
6000/6000 [==============================] - 22s 4ms/sample - loss: 0.2571 - acc: 0.9012 - val_loss: 0.3228 - val_acc: 0.8725
Epoch 3/5
6000/6000 [==============================] - 22s 4ms/sample - loss: 0.1091 - acc: 0.9643 - val_loss: 0.4298 - val_acc: 0.8545
Epoch 4/5
6000/6000 [==============================] - 22s 4ms/sample - loss: 0.0466 - acc: 0.9862 - val_loss: 0.5111 - val_acc: 0.8510 lo
Epoch 5/5
6000/6000 [==============================] - 22s 4ms/sample - loss: 0.0195 - acc: 0.9957 - val_loss: 0.5288 - val_acc: 0.8535


In [14]:
Score_Predict2(df,model_bidir)

MAE1: 0.6345295145344734
Pearson Corr.1 : 0.7849685955260489
Spearman corr.1 : 0.7511649244170083
kendall.1 : 0.5713070767073772
Train on 2000 samples, validate on 6000 samples
Epoch 1/5
 - 1s - loss: 2.7837 - val_loss: 2.2400
Epoch 2/5
 - 0s - loss: 1.7845 - val_loss: 1.2781
Epoch 3/5
 - 0s - loss: 1.0513 - val_loss: 0.8457
Epoch 4/5
 - 0s - loss: 0.7848 - val_loss: 0.6914
Epoch 5/5
 - 0s - loss: 0.6600 - val_loss: 0.6074
MAE2: 0.6835966731414198
Pearson Corr.2 : 0.8023747512708381
Spearman corr.2 : 0.760306070857525
kendall.2 : 0.5503121402659081


In [28]:
import numpy as np 
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import scipy.stats
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
def Score_Predict2(df,model):
    df['score']=df['text'].apply(lambda text: model.predict(pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=max_length, padding=padding_type, truncating=trunc_type)))
    df['My_rate']=df['score'].apply(lambda s: s*4)
  #  df['My_rate']=df['is_humor'].apply(lambda s: [[0]] if s==0 else None)
    df['new_rate'] = df['My_rate'][0][0] - df['humor_controversy']-df['offense_rating']
    df.new_rate.abs()
    df['humor_rating'] = df['humor_rating'].replace(np.nan, 0)
    df['new_rate'] = df['new_rate'].replace(np.nan, 0)
    df['new_rate'] = df['new_rate']*df['is_humor']
    actual = []
    pred = []   
    for i,l,text,humour,rating,controversy,offense,score,my_rate,new_rate in df.itertuples():
            actual.append(rating)
            pred.append(new_rate)
    mae = metrics.mean_absolute_error(actual, pred)
    print("MAE1:",mae)
    print("Pearson Corr.1 :",scipy.stats.pearsonr(actual, pred)[0] )
    print("Spearman corr.1 :",scipy.stats.spearmanr(actual, pred)[0])
    print("kendall.1 :",scipy.stats.kendalltau(actual, pred)[0])
    #2nd method
    features = df[['humor_controversy','offense_rating','score']].values
    labels = df['humor_rating'].values
    # getting training and testing data
    features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .75, random_state = 0)
    df['humor_controversy'] = df['humor_controversy'].replace(np.nan, 0)
    scaler=MinMaxScaler()
    scaler.fit(features_train)
    features_train=scaler.transform(features_train)
    #   print(features_train)
    features_test=scaler.transform(features_test)
    #print(features_test)
    model1=Sequential()
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(4,activation='relu'))
    model1.add(Dense(1))
    model1.compile(loss='mse',optimizer='adam')#,metrics=['accuracy'])
    history=model1.fit(x=features_train,y=labels_train,epochs=5,validation_data=(features_test, labels_test),verbose=2)
    #humor_pred=model1.predict(features_test)
    pred1 = []    
    for i,l,text,humour,rating,controversy,offense,score,my_rate,new_rate in df.itertuples():
            new=[[controversy,offense,score]]
            new=scaler.transform(new)
            pred1.append(model1.predict(new)[0][0])
    #print(pred1)        
    mae2 = metrics.mean_absolute_error(actual, pred1)
    print("MAE2:",mae2)
    print("Pearson Corr.2 :",scipy.stats.pearsonr(actual, pred1)[0] )
    print("Spearman corr.2 :",scipy.stats.spearmanr(actual, pred1)[0])
    print("kendall.2 :",scipy.stats.kendalltau(actual, pred1)[0])


    

In [17]:
model_gru = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GRU(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 64)            1280000   
_________________________________________________________________
gru_1 (GRU)                  (None, 60, 32)            9312      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 1,289,857
Trainable params: 1,289,857
Non-trainable params: 0
_________________________________________________________________


In [21]:
history_gru = model_gru.fit(training_padded, training_labels, batch_size=32, epochs=5, 
                    validation_data=(testing_padded, testing_labels), 
                    callbacks=[rlrp] ,verbose=1)

Train on 6000 samples, validate on 2000 samples
Epoch 1/30
6000/6000 [==============================] - 15s 3ms/sample - loss: 0.0418 - acc: 0.9900 - val_loss: 0.4848 - val_acc: 0.8405
Epoch 2/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0374 - acc: 0.9913 - val_loss: 0.5190 - val_acc: 0.8370
Epoch 3/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0343 - acc: 0.9922 - val_loss: 0.5131 - val_acc: 0.8440
Epoch 4/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0307 - acc: 0.9933 - val_loss: 0.5154 - val_acc: 0.8430
Epoch 5/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0312 - acc: 0.9943 - val_loss: 0.5181 - val_acc: 0.8430
Epoch 6/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0307 - acc: 0.9933 - val_loss: 0.5183 - val_acc: 0.8430
Epoch 7/30
6000/6000 [==============================] - 14s 2ms/sample - loss: 0.0301 - acc: 0.9940 - val_loss: 0.5186 - val_acc: 

In [29]:
Score_Predict2(df,model_gru)

MAE1: 0.637353477563858
Pearson Corr.1 : 0.786476922451849
Spearman corr.1 : 0.7511649244170083
kendall.1 : 0.5713070767073772
Train on 2000 samples, validate on 6000 samples
Epoch 1/5
 - 2s - loss: 2.9844 - val_loss: 2.5435
Epoch 2/5
 - 0s - loss: 1.8582 - val_loss: 1.1701
Epoch 3/5
 - 0s - loss: 0.9035 - val_loss: 0.7494
Epoch 4/5
 - 0s - loss: 0.7217 - val_loss: 0.6664
Epoch 5/5
 - 0s - loss: 0.6437 - val_loss: 0.5932
MAE2: 0.673437413840592
Pearson Corr.2 : 0.8330928855402568
Spearman corr.2 : 0.7635412109353715
kendall.2 : 0.5536798693927716
